In [ ]:
!gdown 16QzKYopDr3-Re1F3Hychv_a19hCDgC8S

Downloading...
From: https://drive.google.com/uc?id=16QzKYopDr3-Re1F3Hychv_a19hCDgC8S
To: /content/twitter-suicidal_data.csv
100% 3.63M/3.63M [00:00<00:00, 144MB/s]


In [ ]:
import pandas as pd
data=pd.read_csv('/content/twitter-suicidal_data.csv')
data.head()


,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [ ]:
import pandas as pd
import re
import spacy

# Load the spaCy English language model
nlp = spacy.load('en_core_web_sm')

# Function to perform text preprocessing
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization (split the text into words)
    words = [token.text for token in nlp(text)]

    # Remove stopwords
    stop_words = set(spacy.lang.en.stop_words.STOP_WORDS)
    words = [word for word in words if word not in stop_words]

    # Join the words back into a string
    preprocessed_text = ' '.join(words)

    return preprocessed_text

# Apply the preprocessing function to the 'text' column
data['preprocessed_text'] = data['tweet'].apply(preprocess_text)




In [ ]:
# Display the preprocessed data
data[['tweet', 'preprocessed_text']].head()

,tweet,preprocessed_text
0,my life is meaningless i just want to end my l...,life meaningless want end life badly life comp...
1,muttering i wanna die to myself daily for a fe...,muttering wanna die daily months feel worthles...
2,work slave i really feel like my only purpose ...,work slave feel like purpose life higher man m...
3,i did something on the 2 of october i overdose...,october overdosed felt horrible hospital da...
4,i feel like no one cares i just want to die ma...,feel like cares want die maybe d feel lonely


In [ ]:
!gdown 0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:21<00:00, 75.9MB/s]


In [ ]:
from gensim.models import KeyedVectors
import numpy as np

# Load pre-trained model
model = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)

# Function to create an embedding matrix
def create_embedding_matrix(model, word_index, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in model:
            embedding_matrix[i] = model[word]
    return embedding_matrix


In [ ]:
texts = data['preprocessed_text'].values

# Tokenize the text
tokenized_texts = [text.split() for text in texts]

# Flatten the list of tokenized texts
flat_tokens = [token for sublist in tokenized_texts for token in sublist]

# Create a set of unique words
unique_words = set(flat_tokens)

# Create a word index
word_index = {word: index + 1 for index, word in enumerate(unique_words)}


# Define embedding dimension based on the pre-trained Word2Vec model
embedding_dim = 300

# Create the embedding matrix
embedding_matrix = create_embedding_matrix(model, word_index, embedding_dim)

# Display the shape of the embedding matrix
print("Shape of the embedding matrix:", embedding_matrix.shape)

In [ ]:
from keras.utils import to_categorical
y= to_categorical(data['intention'].values)

In [ ]:
# Calculate the maximum sequence length in the preprocessed text data
max_len = max(len(seq.split()) for seq in data['preprocessed_text'].values)

# Display the maximum sequence length
print("Max sequence length:", max_len)


Max sequence length: 2021


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Assuming 'embedding_matrix' is already created
embedding_dim = embedding_matrix.shape[1]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['preprocessed_text'], y, test_size=0.2, random_state=42)


# 2-layer LSTM
model_lstm_2layer = Sequential()
model_lstm_2layer.add(Embedding(input_dim=X_train.shape, output_dim=embedding_dim,
                                weights=[embedding_matrix], input_length=max_len, trainable=False))
model_lstm_2layer.add(LSTM(100, return_sequences=True))
model_lstm_2layer.add(LSTM(50))
model_lstm_2layer.add(Dense(2, activation='softmax'))

# Compile the model
model_lstm_2layer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_lstm_2layer.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Single-layer LSTM
model_lstm_single = Sequential()
model_lstm_single.add(Embedding(input_dim=len(model.index_to_key) + 1, output_dim=embedding_dim,
                                weights=[embedding_matrix], input_length=max_len, trainable=False))
model_lstm_single.add(LSTM(100))
model_lstm_single.add(Dense(2, activation='softmax'))

# Compile the model
model_lstm_single.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_lstm_single.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# CNN + 2-layer LSTM
model_cnn_lstm = Sequential()
model_cnn_lstm.add(Embedding(input_dim=len(model.index_to_key) + 1, output_dim=embedding_dim,
                             weights=[embedding_matrix], input_length=max_len, trainable=False))
model_cnn_lstm.add(Conv1D(128, 5, activation='relu'))
model_cnn_lstm.add(MaxPooling1D(pool_size=4))
model_cnn_lstm.add(LSTM(100, return_sequences=True))
model_cnn_lstm.add(LSTM(50))
model_cnn_lstm.add(Dense(2, activation='softmax'))

# Compile the model
model_cnn_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_cnn_lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

# Function to plot the training history
def plot_history(history, model_name):
    # Plot training & validation accuracy values
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# Train and plot for 2-layer LSTM
history_lstm_2layer = model_lstm_2layer.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
plot_history(history_lstm_2layer, '2-layer LSTM')

# Train and plot for Single-layer LSTM
history_lstm_single = model_lstm_single.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
plot_history(history_lstm_single, 'Single-layer LSTM')

# Train and plot for CNN + 2-layer LSTM
history_cnn_lstm = model_cnn_lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
plot_history(history_cnn_lstm, 'CNN + 2-layer LSTM')
